<a href="https://colab.research.google.com/github/Jacob-Francis/SWSG_repo/blob/main/Colab_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [21]:
# Parameters
N = 20
N1 = 22  # testing shapes
M = 25
epsilon = 0.01

# Generate G
G = np.stack([j.flatten() for j in np.meshgrid(
    np.linspace(1/(2*N), 1 - 1/(2*N), N, endpoint=True),
    np.linspace(1/(2*N), 1 - 1/(2*N), N, endpoint=True))], axis=-1)

# Generate X in the desired format (N * N, 2)
X = np.stack([j.flatten() for j in np.meshgrid(
    np.linspace(1/(2*N1), 1 - 1/(2*N1), N, endpoint=True),
    np.linspace(1/(2*N1), 1 - 1/(2*N1), N, endpoint=True))], axis=-1)

# Generate Y in the desired format (M * M, 2)
Y = np.stack([j.flatten() for j in np.meshgrid(
    np.linspace(1/(2*M), 1 - 1/(2*M), M, endpoint=True),
    np.linspace(1/(2*M), 1 - 1/(2*M), M, endpoint=True))], axis=-1)

In [3]:
G.shape

(400, 2)

In [7]:
f1 = np.zeros(N*N)
f2 = np.zeros(N1*N1)
f3 = np.zeros(M*M)

# Compute cost matrices
C_gx = 0.5 * cdist(G, X) ** 2   # Shape (N * N, N * N)
C_xy = 0.5 * cdist(X, Y) ** 2   # Shape (N * N, M * M)

# Expand dimensions to allow broadcasting
C_gx_expanded = C_gx[:, :, np.newaxis]  # Shape (N * N, N * N, 1)
C_xy_expanded = C_xy[np.newaxis, :, :]  # Shape (1, N * N, M * M)

# Stack to form final cost matrix
C = C_gx_expanded + C_xy_expanded

R here is the reference measuer, though everything so far is just uniformly weighted by the number of points

In [24]:
(G[:,  np.newaxis, :] + Y[np.newaxis,  :, :]).shape

(400, 625, 2)

In [26]:
R = 1 / (N**2 * N1**2 * M**2)
f1 = - epsilon * np.log(np.einsum('ijk->i',np.exp((f2[np.newaxis, :, np.newaxis] + f3[np.newaxis, np.newaxis, :] - C) / epsilon)*R))
f2 = - epsilon * np.log(np.einsum('ijk->j',np.exp((f1[:, np.newaxis, np.newaxis] + f3[np.newaxis, np.newaxis, :] - C) / epsilon)*R))
f3 = - epsilon * np.log(np.einsum('ijk->k',np.exp((f2[np.newaxis, :, np.newaxis] + f1[:, np.newaxis, np.newaxis] - C) / epsilon)*R))

# There is a more efficient implementation
X = np.einsum('ijk,ikl->jl', np.exp((f1[:, np.newaxis, np.newaxis] + f2[np.newaxis, :, np.newaxis] + f3[np.newaxis, np.newaxis, :] - C) / epsilon)*R, -G[:,  np.newaxis, :] - Y[np.newaxis,  :, :])